In [1]:
import pandas as pd
import autogluon
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import bisect
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
train_df = pd.read_parquet("data/train_v2.parquet")
test_df = pd.read_parquet("data/test_v2.parquet")

In [3]:
train_data = TabularDataset(train_df)
test_data = TabularDataset(test_df)

label = "CI_HOUR"
eval_metric = "mean_absolute_error"

In [4]:
predictor = TabularPredictor(
    label=label, problem_type='regression', eval_metric=eval_metric,
).fit(train_data, presets="best_quality", num_gpus=1, num_bag_folds=7, num_stack_levels=2)

No path specified. Models will be saved in: "AutogluonModels\ag-20231010_123004\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=7, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20231010_123004\"
AutoGluon Version:  0.8.2
Python Version:     3.9.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   14.79 GB / 498.74 GB (3.0%)
Train Data Rows:    367440
Train Data Columns: 282
Label Column: CI_HOUR
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4090.0 MB
	Train Data (Original)  Memory Usage: 177.84 MB (4.3% of ava

[1000]	valid_set's l1: 53.2643
[2000]	valid_set's l1: 51.3019
[3000]	valid_set's l1: 49.956
[4000]	valid_set's l1: 48.933
[5000]	valid_set's l1: 48.079
[6000]	valid_set's l1: 47.3321
[7000]	valid_set's l1: 46.7781
[8000]	valid_set's l1: 46.2504
[9000]	valid_set's l1: 45.8885
[10000]	valid_set's l1: 45.509
[1000]	valid_set's l1: 52.2301
[2000]	valid_set's l1: 50.5044


KeyboardInterrupt: 

In [5]:
print(predictor.leaderboard(silent=True))

                     model  score_val  pred_time_val      fit_time  \
0      WeightedEnsemble_L3 -18.241295     318.839151  30091.039016   
1     LightGBMLarge_BAG_L2 -18.681279     264.864792   9457.184307   
2           XGBoost_BAG_L2 -19.208710     258.366502  26087.445843   
3          LightGBM_BAG_L2 -19.577228     236.335318   9211.056249   
4    NeuralNetTorch_BAG_L2 -19.960918     209.287764  11834.280827   
5          CatBoost_BAG_L2 -20.172213     207.449161  11107.267524   
6   RandomForestMSE_BAG_L2 -20.271846     238.172094   9639.945336   
7     ExtraTreesMSE_BAG_L2 -20.301756     238.220006   9638.675559   
8   NeuralNetFastAI_BAG_L2 -21.416820     210.006616  10224.348350   
9        LightGBMXT_BAG_L2 -21.427574     220.666533   9076.245043   
10     WeightedEnsemble_L2 -28.325769     100.665054   3465.936976   
11         LightGBM_BAG_L1 -29.398178      29.824250    460.296869   
12    LightGBMLarge_BAG_L1 -30.106108      70.250242    684.092278   
13         CatBoost_

In [6]:
predictor.feature_importance(train_data)

These features in provided data are not utilized by the predictor and will be ignored: ['ARI_PO_CXL1', 'ARI_PO_DEJ2', 'ARI_PO_DIN2', 'ARI_PO_EFG4', 'ARI_PO_GIW5', 'ARI_PO_GQJ7', 'ARI_PO_IVU2', 'ARI_PO_JEA6', 'ARI_PO_KIU2', 'ARI_PO_LHD1', 'ARI_PO_ONW1', 'ARI_PO_RGT8', 'ARI_PO_XGX5', 'ARI_PO_XZF6', 'ARI_PO_ZME5', 'FLAG_Algeria', 'FLAG_Argentina', 'FLAG_Australia', 'FLAG_Azerbaijan', 'FLAG_Bahrain', 'FLAG_Bermuda', 'FLAG_Brazil', 'FLAG_Brunei', 'FLAG_Bulgaria', 'FLAG_Cambodia', 'FLAG_Cameroon', 'FLAG_Chile', 'FLAG_Comoros False', 'FLAG_Cook Islands', 'FLAG_Croatia', 'FLAG_Curacao', 'FLAG_Djibouti', 'FLAG_Dominica', 'FLAG_Egypt', 'FLAG_Equatorial Guinea', 'FLAG_Equatorial Guinea False', 'FLAG_Estonia', 'FLAG_Ethiopia', 'FLAG_Faeroes (Fas)', 'FLAG_Finland', 'FLAG_France', 'FLAG_Gabon', 'FLAG_Gambia', 'FLAG_Georgia', 'FLAG_Ghana', 'FLAG_Guinea-Bissau', 'FLAG_Guyana', 'FLAG_Guyana False', 'FLAG_Honduras', 'FLAG_Irish Republic', 'FLAG_Jordan', 'FLAG_Kiribati', 'FLAG_Kuwait', 'FLAG_Latvia', 'FL

,importance,stddev,p_value,n,p99_high,p99_low
WTI,9.335927e+02,8.892089e+00,9.874479e-10,5,951.901645,915.283775
DUBAI,8.477199e+02,6.592861e+00,4.389674e-10,5,861.294716,834.145134
BRENT,8.380975e+02,8.634389e+00,1.351661e-09,5,855.875866,820.319214
month,6.690862e+02,8.742037e+00,3.496268e-09,5,687.086220,651.086268
BDI_ADJ,6.536173e+02,9.385299e+00,5.099882e-09,5,672.941755,634.292835
...,...,...,...,...,...,...
FLAG_Israel,1.560934e-07,7.714369e-07,3.371992e-01,5,0.000002,-0.000001
FLAG_Philippines,-3.944276e-05,2.432199e-03,5.135946e-01,5,0.004968,-0.005047
ARI_PO_PUF3,-1.957740e-04,4.889346e-03,5.335194e-01,5,0.009871,-0.010263
FLAG_Thailand,-2.649475e-04,2.310960e-03,5.948415e-01,5,0.004493,-0.005023


In [7]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

In [8]:
model_pred

0           95.837402
1          390.109039
2            0.018952
3            0.149630
4           31.008961
             ...     
244984      93.460342
244985     331.323547
244986      -0.031149
244987      -0.129224
244988    1315.939575
Name: CI_HOUR, Length: 244989, dtype: float32

In [9]:
submit = pd.read_csv("data/sample_submission.csv")
submit["CI_HOUR"] = [i if i > 0 else 0.0 for i in model_pred]
submit.to_csv("./csv/autogluon_v1.csv", index=False)